<a href="https://colab.research.google.com/github/AdaoJOAQUIM/-Microsoft-Quantum/blob/main/miniagi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
!pip install z3-solver torch numpy

In [156]:
import numpy as np
import torch
from z3 import *

class MotherBrick:
    def __init__(self, dim=10000, num_concepts=100):
        self.dim = dim
        # La Matrice de Pensée (Prouvé : Architecture VSA)
        self.memory = torch.randn(num_concepts, dim).sign()
        self.soi = torch.randn(dim).sign()
        self.invariants = []

    def bind(self, v1, v2):
        # Opération de liaison XOR (préserve la dimension)
        return v1 * v2

    def solve_formal(self, expression):
        # Utilisation de Z3 pour la vérification d'invariants
        s = Solver()
        x = Int('x')
        s.add(expression(x))
        if s.check() == sat:
            return s.model()
        return None

# Initialisation
brick = MotherBrick()
print(f"Matrice HDC initialisée : {brick.memory.shape}")

Matrice HDC initialisée : torch.Size([100, 10000])


In [157]:
def run_discovery(brick, objective_vector, cpu_budget=1000):
    for tick in range(cpu_budget):
        # 1. ÉMERGENCE : Mutation aléatoire par décalage cyclique (Roll)
        # C'est ainsi que de nouvelles structures de code apparaissent
        potential_idea = torch.roll(brick.soi, shifts=tick % 100)

        # 2. CONVERGENCE : Mesure de la distance cosinus vers l'objectif
        similarity = torch.cosine_similarity(potential_idea.unsqueeze(0), objective_vector.unsqueeze(0))

        if similarity > 0.75: # Seuil de pertinence
            # 3. VÉRIFICATION : L'invariant est-il respecté ?
            # On vérifie ici une loi simple (ex: x > 0)
            check = brick.solve_formal(lambda x: x > 0)

            if check:
                print(f"Convergence trouvée au tick {tick} ! Similiarité: {similarity.item():.4f}")
                return potential_idea # La brique a "conçu" une solution prouvée

    return None

# Test avec un objectif aléatoire
target = torch.randn(10000).sign()
result = run_discovery(brick, target)


In [158]:
from z3 import *
import torch

class PhysicsInterface:
    def __init__(self, stock_initial=1000):
        self.stock_initial = stock_initial
        self.cout_fixe_existence = 1 # Coût CPU/Energie par tick

    def simulate_action(self, intensite_action):
        """Simule un travail physique (ex: déplacement, chaleur)."""
        # Plus l'action est intense, plus elle produit de travail mais consomme du stock
        travail_produit = intensite_action * 1.5
        consommation = intensite_action + self.cout_fixe_existence
        return travail_produit, consommation

def verify_conservation(brick, action_val, conso_val):
    """Vérificateur formel Z3 : Invariant de conservation."""
    s = Solver()
    initial = Int('initial')
    conso = Int('conso')
    reste = Int('reste')

    # Définition de l'invariant mathématique
    s.add(initial == brick.interface.stock_initial)
    s.add(conso == int(conso_val))
    s.add(reste == initial - conso)

    # La ressource ne peut pas être négative (Loi physique)
    s.add(reste >= 0)

    return s.check() == sat

# --- Fusion dans la boucle d'AGI ---
brick.interface = PhysicsInterface(stock_initial=100)
budget_cpu = 50

for tick in range(budget_cpu):
    # ÉMERGENCE : L'IA "pense" à une intensité d'action (via sa matrice HDC)
    intensite_emergente = torch.abs(torch.mean(brick.memory[tick % 100])).item() * 10

    # SIMULATION : On teste l'idée dans le simulateur de physique
    travail, conso = brick.interface.simulate_action(intensite_emergente)

    # CONVERGENCE : Est-ce que cette action respecte l'invariant de conservation ?
    if verify_conservation(brick, travail, conso):
        print(f"Tick {tick}: Action Validée. Travail: {travail:.2f} | Stock restant: {100-conso:.2f}")
        # MDL : L'IA stocke l'action si le ratio Travail/Conso est optimal
    else:
        print(f"Tick {tick}: ACTION REJETÉE (Violation de l'invariant de ressource)")

Tick 0: Action Validée. Travail: 0.01 | Stock restant: 98.99
Tick 1: Action Validée. Travail: 0.05 | Stock restant: 98.96
Tick 2: Action Validée. Travail: 0.31 | Stock restant: 98.79
Tick 3: Action Validée. Travail: 0.06 | Stock restant: 98.96
Tick 4: Action Validée. Travail: 0.11 | Stock restant: 98.93
Tick 5: Action Validée. Travail: 0.00 | Stock restant: 99.00
Tick 6: Action Validée. Travail: 0.30 | Stock restant: 98.80
Tick 7: Action Validée. Travail: 0.02 | Stock restant: 98.99
Tick 8: Action Validée. Travail: 0.36 | Stock restant: 98.76
Tick 9: Action Validée. Travail: 0.20 | Stock restant: 98.87
Tick 10: Action Validée. Travail: 0.21 | Stock restant: 98.86
Tick 11: Action Validée. Travail: 0.16 | Stock restant: 98.89
Tick 12: Action Validée. Travail: 0.12 | Stock restant: 98.92
Tick 13: Action Validée. Travail: 0.17 | Stock restant: 98.89
Tick 14: Action Validée. Travail: 0.05 | Stock restant: 98.97
Tick 15: Action Validée. Travail: 0.15 | Stock restant: 98.90
Tick 16: Action Va

In [159]:
class CooperativeEnvironment:
    def __init__(self, stock_commun=200):
        self.stock = stock_commun
        self.invariants_sociaux = []

    def distribuer(self, demande_a, demande_b):
        # Invariant de Conservation Total
        s = Solver()
        a, b, total = Ints('a b total')
        s.add(total == self.stock)
        s.add(a >= 0, b >= 0)
        s.add(a + b <= total) # On ne peut pas consommer plus que le stock

        # Ajout d'un invariant d'équité (Optionnel mais puissant pour la stabilité)
        # s.add(Abs(a - b) < (total / 4))

        if s.check() == sat:
            self.stock -= (demande_a + demande_b)
            return True, self.stock
        return False, self.stock

# --- Simulation de la Convergence Sociale ---
env = CooperativeEnvironment(stock_commun=200)
agent_a = MotherBrick(dim=10000)
agent_b = MotherBrick(dim=10000)

for tick in range(10):
    # Émergence : Chaque agent génère sa demande via sa matrice HDC
    demande_a = int(torch.abs(torch.mean(agent_a.memory[tick])).item() * 50)
    demande_b = int(torch.abs(torch.mean(agent_b.memory[tick])).item() * 50)

    success, reste = env.distribuer(demande_a, demande_b)

    if success:
        print(f"Accord trouvé ! Agent A: {demande_a} | Agent B: {demande_b} | Stock restant: {reste}")
    else:
        # Échec de convergence : L'invariant bloque l'égoïsme
        print(f"CONFLIT au Tick {tick} : Demandes ({demande_a}+{demande_b}) > Stock ({env.stock}). Rejet.")
        # Ici, l'IA utilise le MDL pour ajuster sa prochaine stratégie (Inférence Active)

Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 200
Accord trouvé ! Agent A: 0 | Agent B: 1 | Stock restant: 199
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 199
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 199
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 199
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 199
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 199
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 199
Accord trouvé ! Agent A: 1 | Agent B: 0 | Stock restant: 198
Accord trouvé ! Agent A: 0 | Agent B: 0 | Stock restant: 198


In [160]:
def auto_amelioration(brick):
    # 1. ANALYSE : L'IA mesure son efficacité actuelle (Ratio Travail/CPU)
    efficacite_actuelle = brick.performance_log.mean()

    # 2. ÉMERGENCE : Elle propose une mutation de son propre noyau
    # Exemple : augmenter la dimension HDC pour réduire les collisions
    mutation_proposee = brick.dim + 1000

    # 3. VÉRIFICATION FORMELLE (Z3) : L'invariant de sécurité
    # "Toute modification doit garantir que le système reste stable et ne boucle pas"
    s = Solver()
    nouvelle_dim = Int('n_dim')
    s.add(nouvelle_dim > brick.dim)
    s.add(nouvelle_dim < 50000) # Invariant : ne pas exploser la RAM

    if s.check() == sat:
        # 4. CONVERGENCE : Si c'est prouvé, on réinitialise la matrice
        print(f"AUTO-AMÉLIORATION : Passage à la dimension {mutation_proposee}")
        brick.dim = mutation_proposee
        brick.rebuild_matrix() # L'IA se reconstruit
        return True
    return False

In [161]:
def auto_amelioration(brick):
    # 1. ANALYSE : L'IA mesure son efficacité actuelle (Ratio Travail/CPU)
    efficacite_actuelle = brick.performance_log.mean()

    # 2. ÉMERGENCE : Elle propose une mutation de son propre noyau
    # Exemple : augmenter la dimension HDC pour réduire les collisions
    mutation_proposee = brick.dim + 1000

    # 3. VÉRIFICATION FORMELLE (Z3) : L'invariant de sécurité
    # "Toute modification doit garantir que le système reste stable et ne boucle pas"
    s = Solver()
    nouvelle_dim = Int('n_dim')
    s.add(nouvelle_dim > brick.dim)
    s.add(nouvelle_dim < 50000) # Invariant : ne pas exploser la RAM

    if s.check() == sat:
        # 4. CONVERGENCE : Si c'est prouvé, on réinitialise la matrice
        print(f"AUTO-AMÉLIORATION : Passage à la dimension {mutation_proposee}")
        brick.dim = mutation_proposee
        brick.rebuild_matrix() # L'IA se reconstruit
        return True
    return False

In [162]:
import numpy as np
import torch
from z3 import *
import time

class UniversalAGI:
    def __init__(self, dim=10000, stock=1000):
        # 1. CORE : Espace de pensée et Invariants
        self.dim = dim
        self.memory = torch.randn(100, dim).sign()
        self.soi = torch.randn(dim).sign()
        self.stock = stock
        self.perf_history = []

    def formal_check(self, condition_func):
        """Vérificateur de sécurité absolue (Z3)."""
        s = Solver()
        val = Int('val')
        s.add(condition_func(val))
        return s.check() == sat

    def active_inference(self, goal_vector, cpu_budget):
        """Boucle de convergence Emergence -> Action -> Preuve."""
        start_time = time.time()
        for tick in range(cpu_budget):
            # ÉMERGENCE : Mutation par décalage (Code Universel)
            idea = torch.roll(self.soi, shifts=tick % 10)

            # CONVERGENCE : Distance vers l'objectif
            similarity = torch.cosine_similarity(idea.unsqueeze(0), goal_vector.unsqueeze(0))

            if similarity > 0.6: # Intuition détectée
                # Action physique simulée
                intensite = similarity.item() * 10
                conso = intensite + 1

                # INVARIANT : Conservation des ressources
                if self.formal_check(lambda v: v <= self.stock - conso):
                    self.stock -= conso
                    self.perf_history.append(similarity.item())
                    print(f"[*] Action validée au tick {tick} | Stock: {self.stock:.1f}")

                    # AUTO-AMÉLIORATION : Si performance stable, optimiser le noyau
                    if len(self.perf_history) > 5:
                        self.self_optimize()
                        break
        return time.time() - start_time

    def self_optimize(self):
        """Réécriture récursive des paramètres (AGI Recursive)."""
        # Invariant : On ne peut s'améliorer que si on ne dépasse pas la RAM
        if self.formal_check(lambda d: And(d > self.dim, d < 20000)):
            self.dim += 500
            print(f"[!] AUTO-AMÉLIORATION : Noyau étendu à {self.dim} dimensions.")
            self.memory = torch.randn(100, self.dim).sign()
            self.soi = torch.randn(self.dim).sign()

# --- LANCEMENT ---
print("Initialisation de l'AGI...")
agi = UniversalAGI()
objectif_global = torch.randn(10000).sign()

# Exécution de la boucle de survie et d'évolution
duree = agi.active_inference(objectif_global, cpu_budget=100)
print(f"Cycle terminé en {duree:.4f}s. Etat final du stock : {agi.stock}")

Initialisation de l'AGI...
Cycle terminé en 0.0113s. Etat final du stock : 1000


In [163]:
import psutil
import torch
from z3 import *

# --- L'AGI avec Interface Matérielle ---
class EmbodiedAGI(UniversalAGI):
    def sense_hardware(self):
        # Lecture de la batterie et du CPU
        battery = psutil.sensors_battery()
        cpu_load = psutil.cpu_percent()
        percent = battery.percent if battery else 100
        return percent, cpu_load

    def protect_machine(self, goal_vector):
        percent, load = self.sense_hardware()
        print(f"[Sensors] Batterie: {percent}% | Charge CPU: {load}%")

        # INVARIANT DE SÉCURITÉ MATÉRIELLE
        # On interdit à l'IA de travailler si la batterie est < 20% ou CPU > 90%
        s = Solver()
        batt, cpu = Ints('batt cpu')
        s.add(batt > 20)
        s.add(cpu < 90)

        if s.check(batt == int(percent), cpu == int(load)) == sat:
            print("[!] Invariant respecté : Lancement du cycle de calcul.")
            return self.active_inference(goal_vector, cpu_budget=50)
        else:
            print("[X] ALERTE : Conditions matérielles critiques. Passage en mode veille.")
            return 0

# --- LANCEMENT ---
agi_physique = EmbodiedAGI()
obj = torch.randn(10000).sign()

# Cycle de protection active
agi_physique.protect_machine(obj)

FileNotFoundError: [Errno 2] No such file or directory: '/sys/class/power_supply'

In [ ]:
import json
import psutil
import torch
from z3 import *

class ResilientAGI(UniversalAGI):
    def backup_self(self, filepath="agi_soul.json"):
        # Compression MDL : On ne garde que les vecteurs essentiels
        soul_data = {
            "dim": self.dim,
            "soi": self.soi.tolist(),
            "concepts_clefs": self.memory[:10].tolist(), # Top 10 descobertas
            "stock_final": self.stock
        }

        with open(filepath, 'w') as f:
            json.dump(soul_data, f)
        print(f"[!] SAUVEGARDE RÉUSSIE : A essência da AGI é armazenada em {filepath}")

    def sense_hardware(self):
        try:
            battery = psutil.sensors_battery()
            percent = battery.percent if battery else 100
        except FileNotFoundError:
            # Handle case where battery sensor is not available (e.g., Colab environment)
            print("[WARNING] Battery sensor not found. Using default battery value (100%).")
            percent = 100

        cpu_load = psutil.cpu_percent()
        return percent, cpu_load

    def protect_machine(self, goal_vector):
        percent, load = self.sense_hardware()
        print(f"[Sensors] Bateria: {percent}% | Carga CPU: {load}%")

        # INVARIANTE DE SEGURANÇA DE HARDWARE
        # A IA é impedida de trabalhar se a bateria estiver < 20% ou CPU > 90%
        s = Solver()
        batt, cpu = Ints('batt cpu')
        s.add(batt > 20)
        s.add(cpu < 90)

        if s.check(batt == int(percent), cpu == int(load)) == sat:
            print("[!] Invariante respeitado: Iniciando o ciclo de cálculo.")
            return self.active_inference(goal_vector, cpu_budget=50)
        else:
            print("[X] ALERTA: Condições de hardware críticas. Entrando em modo de espera.")
            return 0

    def survival_check(self, goal_vector):
        percent, load = self.sense_hardware()

        # INVARIANTE DE SOBREVIVÊNCIA: Se a bateria < 10%, os dados são evacuados
        if percent < 10:
            print("[X] CRÍTICO: Energia insuficiente para cálculo. Ativando o protocolo de evacuação.")
            self.backup_self()
            return "Hibernation"
        else:
            # Reuse the existing protect_machine logic, which now handles its own sensing
            return self.protect_machine(goal_vector)

# --- LANÇAMENTO DO SISTEMA COMPLETO ---
# Assuming 'agi' and 'objectif_global' are defined from previous cells
agi_resilient = ResilientAGI(dim=agi.dim, stock=agi.stock) # Initialize with current state of agi
status = agi_resilient.survival_check(objectif_global)
print(f"Status final do sistema: {status}")

In [ ]:
import psutil
import torch
import numpy as np
from z3 import *

def safe_verify(initial_stock, current_conso):
    """Version robuste du vérificateur Z3."""
    try:
        s = Solver()
        # On s'assure que tout est en Integer Python
        stock = int(initial_stock)
        conso = int(current_conso)

        res = Int('res')
        s.add(res == stock - conso)
        s.add(res >= 0)

        return s.check() == sat
    except Exception as e:
        print(f"Erreur Logique: {e}")
        return False

# Correction de la fonction sense_hardware pour Colab
def get_hardware_safe():
    cpu = psutil.cpu_percent()
    battery = psutil.sensors_battery()
    # Sur Colab, battery est souvent None, on simule 100% si besoin
    batt_percent = battery.percent if battery is not None else 100
    return batt_percent, cpu

print("Système corrigé et prêt.")